In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.api import add_constant
from statsmodels.sandbox.regression.gmm import IV2SLS

%store -r full
%store -r solution

In [5]:
full

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
0,0,lemartinp,3,270.84,1,0.0,0.0,4.0,1.0,0.146341,0.136576
1,1,juligarji,3,167.50,1,4.0,4.0,5.0,3.0,0.057325,0.055742
2,2,nfmorenog,3,225.00,1,0.0,2.0,3.0,1.0,0.190476,0.174353
3,3,AFelipeGA,3,291.67,1,4.0,3.0,5.0,2.0,0.090909,0.087011
4,4,mballeng91,3,329.17,1,2.0,6.0,6.0,2.0,0.004854,0.004843
...,...,...,...,...,...,...,...,...,...,...,...
159,159,NicolasPrr,3,237.50,4,3.0,4.0,3.0,2.0,0.009360,0.009316
160,160,jumcorredorro,3,53.33,4,1.0,1.0,5.0,2.0,0.013294,0.013206
161,161,feartheGru,3,358.33,4,2.0,2.0,5.0,2.0,0.000000,0.000000
162,162,Danielsv9207,3,358.33,4,1.0,1.0,1.0,3.0,0.000000,0.000000


In [6]:
solution

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
1,1,bdvegat,2,260.00,1,1.0,2.0,4.0,1.0,0.054187,0.052770
2,2,Phoenixest,2,185.83,1,0.0,1.0,3.0,1.0,0.017964,0.017805
3,3,HashNick,2,260.00,1,1.0,3.0,5.0,3.0,0.023697,0.023420
4,4,ccvacad,2,260.00,1,1.0,0.0,4.0,2.0,0.000000,0.000000
5,5,joaortizro,2,152.50,1,0.0,2.0,2.0,1.0,0.041420,0.040585
...,...,...,...,...,...,...,...,...,...,...,...
162,162,lsfinite,2,358.33,4,1.0,1.0,6.0,3.0,0.008392,0.008357
163,163,xdanielsb,2,283.33,4,3.0,3.0,3.0,2.0,0.040128,0.039344
164,164,CSebasGomez,2,358.33,4,3.0,3.0,5.0,2.0,0.000000,0.000000
165,165,jhcardenasa,2,358.33,4,4.0,4.0,5.0,3.0,0.051020,0.049762


In [9]:
df = solution.append(full, ignore_index=True)
df

,Unnamed: 0,Student,group,abs_perform_diff_best,phase,Q7_Q7_1,Q7_Q7_2,Q8_Q8_1,Q10,similarity,ln_similarity
0,1,bdvegat,2,260.00,1,1.0,2.0,4.0,1.0,0.054187,0.052770
1,2,Phoenixest,2,185.83,1,0.0,1.0,3.0,1.0,0.017964,0.017805
2,3,HashNick,2,260.00,1,1.0,3.0,5.0,3.0,0.023697,0.023420
3,4,ccvacad,2,260.00,1,1.0,0.0,4.0,2.0,0.000000,0.000000
4,5,joaortizro,2,152.50,1,0.0,2.0,2.0,1.0,0.041420,0.040585
...,...,...,...,...,...,...,...,...,...,...,...
311,159,NicolasPrr,3,237.50,4,3.0,4.0,3.0,2.0,0.009360,0.009316
312,160,jumcorredorro,3,53.33,4,1.0,1.0,5.0,2.0,0.013294,0.013206
313,161,feartheGru,3,358.33,4,2.0,2.0,5.0,2.0,0.000000,0.000000
314,162,Danielsv9207,3,358.33,4,1.0,1.0,1.0,3.0,0.000000,0.000000


In [16]:
#Let's confirm that aspiration satisfy the relevance condition for performance distance to the best. There is a difference between aspiration and social aspiration. 
reg_expr = 'abs_perform_diff_best ~ group'

# Build and train an OLS model that regresses performance distance to the best on aspiration and verify
# using the F-test that coefficients of aspiration is significant 
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

df['ln_similarity'] = np.log(df['similarity'] + 1)

# Build out the exog matrix. Statsmodels requires this matrix to contain all the endogenous and
# exogenous variables, plus the constant.
exog = df[['abs_perform_diff_best']]
exog = add_constant(exog)

# Build out the instruments matrix. Statsmodels requires this matrix to contain not only all the
# instruments but also the variables in exog that will NOT be instrumented
instruments = df[['group']]
instruments = add_constant(instruments)

#Build and train the IV2SLS model
iv2sls_model = IV2SLS(endog=df['ln_similarity'], exog=exog, instrument=instruments)
iv2sls_model_results = iv2sls_model.fit()

#Print the training summary
print(iv2sls_model_results.summary())

#Compare the performance of 2SLS with OLS of ln(wage) on performance distance to the best
reg_expr = 'ln_similarity ~ abs_perform_diff_best + group'
olsr_model = smf.ols(formula=reg_expr, data=df)
olsr_model_results = olsr_model.fit()
print(olsr_model_results.summary())

                              OLS Regression Results                             
Dep. Variable:     abs_perform_diff_best   R-squared:                       0.004
Model:                               OLS   Adj. R-squared:                  0.001
Method:                    Least Squares   F-statistic:                     1.257
Date:                   Fri, 16 Sep 2022   Prob (F-statistic):              0.263
Time:                           22:05:10   Log-Likelihood:                -1837.9
No. Observations:                    316   AIC:                             3680.
Df Residuals:                        314   BIC:                             3687.
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    171.2773 